In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Lambda, Dense, Normalization, Dropout
from tensorflow.keras.regularizers import l1_l2, l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

import keras.backend as K
from keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv('Data/dataset1.csv')
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

scaler = MinMaxScaler((0.01, 0.99))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [3]:
mu = X_train.min(axis=0) * 0.99 / 0.98 - 0.01 / 0.98 * X_train.max(axis=0)
sigma = (X_train.max(axis=0) - X_train.min(axis=0)) / 0.98

In [4]:
np.savetxt('Outputs/wong-howard-1.txt', np.vstack([mu.values, sigma.values]))

In [5]:
input_shape = (X_train_scaled.shape[1],)
inputs = Input(input_shape)

x = Lambda(lambda x: tf.concat([
    x[:, :2],
    tf.expand_dims(K.log(x[:, 2]), axis=1),
    tf.expand_dims(x[:, 3], axis=1),
    tf.expand_dims(K.log(x[:, 4]), axis=1),
    tf.expand_dims(K.sqrt(x[:, 5]), axis=1),
], axis=1))(inputs)

x = Normalization()(x)
x = Dense(64, activation='relu', kernel_regularizer=l1_l2(1e-4, 1e-3))(inputs)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(2, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])
es = EarlyStopping('val_acc', patience=250, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train, validation_split=0.3, shuffle=True,
                    batch_size=32, epochs=10000, callbacks=[es])

Epoch 1/10000
35/35 [==============================] - 1s 7ms/step - loss: 0.7191 - acc: 0.4777 - val_loss: 0.7138 - val_acc: 0.4583
Epoch 2/10000
35/35 [==============================] - 0s 3ms/step - loss: 0.7111 - acc: 0.5116 - val_loss: 0.7062 - val_acc: 0.4729
Epoch 3/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.6990 - acc: 0.5732 - val_loss: 0.6982 - val_acc: 0.5542
Epoch 4/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.6942 - acc: 0.6036 - val_loss: 0.6874 - val_acc: 0.6833
Epoch 5/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.6813 - acc: 0.6339 - val_loss: 0.6720 - val_acc: 0.7854
Epoch 6/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.6683 - acc: 0.6670 - val_loss: 0.6509 - val_acc: 0.8354
Epoch 7/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.6548 - acc: 0.7009 - val_loss: 0.6268 - val_acc: 0.8229
Epoch 8/10000
35/35 [==============================] - 0s 5ms/step - 

35/35 [==============================] - 0s 5ms/step - loss: 0.2137 - acc: 0.9179 - val_loss: 0.2114 - val_acc: 0.9292
Epoch 124/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.2201 - acc: 0.9089 - val_loss: 0.2125 - val_acc: 0.9271
Epoch 125/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.2052 - acc: 0.9143 - val_loss: 0.2102 - val_acc: 0.9292
Epoch 126/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.2094 - acc: 0.9134 - val_loss: 0.2137 - val_acc: 0.9229
Epoch 127/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.2178 - acc: 0.9107 - val_loss: 0.2120 - val_acc: 0.9271
Epoch 128/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1997 - acc: 0.9232 - val_loss: 0.2172 - val_acc: 0.9208
Epoch 129/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.2048 - acc: 0.9170 - val_loss: 0.2132 - val_acc: 0.9271
Epoch 130/10000
35/35 [==============================] - 0s 5ms/step - 

Epoch 184/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1944 - acc: 0.9223 - val_loss: 0.2051 - val_acc: 0.9312
Epoch 185/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1961 - acc: 0.9259 - val_loss: 0.2066 - val_acc: 0.9271
Epoch 186/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1949 - acc: 0.9232 - val_loss: 0.2067 - val_acc: 0.9250
Epoch 187/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1987 - acc: 0.9223 - val_loss: 0.2026 - val_acc: 0.9333
Epoch 188/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1968 - acc: 0.9170 - val_loss: 0.2035 - val_acc: 0.9292
Epoch 189/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.2056 - acc: 0.9152 - val_loss: 0.2063 - val_acc: 0.9292
Epoch 190/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1968 - acc: 0.9161 - val_loss: 0.2008 - val_acc: 0.9292
Epoch 191/10000
35/35 [==============================] 

35/35 [==============================] - 0s 5ms/step - loss: 0.1780 - acc: 0.9295 - val_loss: 0.1978 - val_acc: 0.9396
Epoch 305/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1784 - acc: 0.9277 - val_loss: 0.1959 - val_acc: 0.9375
Epoch 306/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1716 - acc: 0.9304 - val_loss: 0.2000 - val_acc: 0.9417
Epoch 307/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1741 - acc: 0.9295 - val_loss: 0.2003 - val_acc: 0.9417
Epoch 308/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1792 - acc: 0.9304 - val_loss: 0.1952 - val_acc: 0.9375
Epoch 309/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1837 - acc: 0.9259 - val_loss: 0.1971 - val_acc: 0.9375
Epoch 310/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1757 - acc: 0.9259 - val_loss: 0.1978 - val_acc: 0.9375
Epoch 311/10000
35/35 [==============================] - 0s 5ms/step - 

Epoch 365/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1678 - acc: 0.9330 - val_loss: 0.1945 - val_acc: 0.9333
Epoch 366/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.1770 - acc: 0.9286 - val_loss: 0.1974 - val_acc: 0.9333
Epoch 367/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1681 - acc: 0.9304 - val_loss: 0.1987 - val_acc: 0.9417
Epoch 368/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1652 - acc: 0.9348 - val_loss: 0.1954 - val_acc: 0.9396
Epoch 369/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1756 - acc: 0.9268 - val_loss: 0.2062 - val_acc: 0.9458
Epoch 370/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1754 - acc: 0.9304 - val_loss: 0.1996 - val_acc: 0.9354
Epoch 371/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1726 - acc: 0.9295 - val_loss: 0.2014 - val_acc: 0.9375
Epoch 372/10000
35/35 [==============================] 

Epoch 486/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.1647 - acc: 0.9339 - val_loss: 0.1958 - val_acc: 0.9396
Epoch 487/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1620 - acc: 0.9339 - val_loss: 0.1973 - val_acc: 0.9396
Epoch 488/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1607 - acc: 0.9277 - val_loss: 0.2000 - val_acc: 0.9396
Epoch 489/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1723 - acc: 0.9295 - val_loss: 0.1978 - val_acc: 0.9417
Epoch 490/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1595 - acc: 0.9402 - val_loss: 0.1951 - val_acc: 0.9375
Epoch 491/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1717 - acc: 0.9339 - val_loss: 0.2004 - val_acc: 0.9438
Epoch 492/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1611 - acc: 0.9402 - val_loss: 0.1992 - val_acc: 0.9458
Epoch 493/10000
35/35 [==============================] 

Epoch 607/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1743 - acc: 0.9295 - val_loss: 0.1983 - val_acc: 0.9438
Epoch 608/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.1631 - acc: 0.9339 - val_loss: 0.1973 - val_acc: 0.9375
Epoch 609/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.1545 - acc: 0.9384 - val_loss: 0.1996 - val_acc: 0.9396
Epoch 610/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1592 - acc: 0.9348 - val_loss: 0.1978 - val_acc: 0.9312
Epoch 611/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1624 - acc: 0.9286 - val_loss: 0.1985 - val_acc: 0.9354
Epoch 612/10000
35/35 [==============================] - 0s 5ms/step - loss: 0.1587 - acc: 0.9348 - val_loss: 0.1989 - val_acc: 0.9375
Epoch 613/10000
35/35 [==============================] - 0s 4ms/step - loss: 0.1570 - acc: 0.9384 - val_loss: 0.1981 - val_acc: 0.9333
Epoch 614/10000
35/35 [==============================] 

In [6]:
model.evaluate(X_test_scaled, y_test)

13/13 [==============================] - 0s 5ms/step - loss: 0.2494 - acc: 0.9175


[0.24938558042049408, 0.9175000190734863]

In [8]:
model.save('Outputs/wong-howard-1.h5')